In [22]:
import pandas as pd 
import requests 
import yfinance as yf 

import os 
import sys


In [28]:
import tickers 

## Extract

In [4]:
def get_income_statement(tickers, period='annually'):
    """
    Fetch income statement data for a list of tickers using yfinance.

    Args:
    tickers (str or list): A single stock ticker symbol or a list of stock ticker symbols.
    period (str): 'annually' or 'quarterly' to fetch annual or quarterly income statement data.

    Returns:
    pandas.DataFrame: A DataFrame containing the income statement data for all tickers.
    """

    all_data = []

    # Ensure tickers is a list
    if isinstance(tickers, str):
        tickers = [tickers]

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        if period == 'annually':
            income_statement = ticker.financials if ticker.financials is not None else pd.DataFrame()
        elif period == 'quarterly':
            income_statement = ticker.quarterly_financials if ticker.quarterly_financials is not None else pd.DataFrame()
        else:
            raise ValueError("Invalid period. Use 'annually' or 'quarterly'.")

        if not income_statement.empty:
            # Standardize date format
            if period == 'annually':
                income_statement.columns = pd.to_datetime(income_statement.columns).to_period('Y').to_timestamp('Y')
            else:
                income_statement.columns = pd.to_datetime(income_statement.columns).to_period('Q').to_timestamp('Q')
            
            # Transpose the DataFrame
            income_statement = income_statement.T
            
            # Reset index to make date a column
            income_statement.reset_index(inplace=True)
            income_statement.rename(columns={'index': 'Date'}, inplace=True)
            
            # Add ticker column
            income_statement['Symbol'] = ticker_symbol
            
            if period == 'annually':
                # Filter for the last 4 years
                four_years_ago = pd.Timestamp.now() - pd.DateOffset(years=4)
                income_statement = income_statement[income_statement['Date'] >= four_years_ago]
            elif period == 'quarterly':
                # Filter for the last 8 quarters
                eight_quarters_ago = pd.Timestamp.now() - pd.DateOffset(months=24)
                income_statement = income_statement[income_statement['Date'] >= eight_quarters_ago]
            
            all_data.append(income_statement)

    # Concatenate all DataFrames into a single DataFrame
    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        
        # Include only the specified columns
        columns_to_include = ['Date', 'Symbol', 'Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
                              'Normalized EBITDA',
                              'Net Income From Continuing Operation Net Minority Interest',
                              'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
                              'EBIT', 'Net Interest Income', 'Interest Expense', 'Normalized Income',
                              'Net Income From Continuing And Discontinued Operation',
                              'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
                              'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
                              'Net Income Common Stockholders', 'Net Income',
                              'Net Income Including Noncontrolling Interests',
                              'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
                              'Net Non Operating Interest Income Expense',
                              'Interest Expense Non Operating', 'Operating Income',
                              'Operating Expense', 'Research And Development',
                              'Selling General And Administration', 'Gross Profit', 'Cost Of Revenue',
                              'Total Revenue', 'Operating Revenue']
        result_df = result_df[columns_to_include]
        
        # Reorder columns to have Date and Symbol first
        cols = ['Date', 'Symbol'] + [col for col in result_df.columns if col not in ['Date', 'Symbol']]
        result_df = result_df[cols]
        
        # Sort by Date (descending) and Symbol
        result_df = result_df.sort_values(['Symbol', 'Date'], ascending=[True, False])
    else:
        result_df = pd.DataFrame()

    return result_df

In [5]:
def get_balance_sheet(tickers, period='annually'):
    """
    Fetch balance sheet data for a list of tickers using yfinance.

    Args:
    tickers (str or list): A single stock ticker symbol or a list of stock ticker symbols.
    period (str): 'annually' or 'quarterly' to fetch annual or quarterly balance sheet data.

    Returns:
    pandas.DataFrame: A DataFrame containing the balance sheet data for all tickers.
    """

    all_data = []

    # Ensure tickers is a list
    if isinstance(tickers, str):
        tickers = [tickers]

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        if period == 'annually':
            balance_sheet = ticker.balance_sheet if ticker.balance_sheet is not None else pd.DataFrame()
        elif period == 'quarterly':
            balance_sheet = ticker.quarterly_balance_sheet if ticker.quarterly_balance_sheet is not None else pd.DataFrame()
        else:
            raise ValueError("Invalid period. Use 'annually' or 'quarterly'.")

        if not balance_sheet.empty:
            # Standardize date format
            if period == 'annually':
                balance_sheet.columns = pd.to_datetime(balance_sheet.columns).to_period('Y').to_timestamp('Y')
            else:
                balance_sheet.columns = pd.to_datetime(balance_sheet.columns).to_period('Q').to_timestamp('Q')
            
            # Transpose the DataFrame
            balance_sheet = balance_sheet.T
            
            # Reset index to make date a column
            balance_sheet.reset_index(inplace=True)
            balance_sheet.rename(columns={'index': 'Date'}, inplace=True)
            
            # Add ticker column
            balance_sheet['Symbol'] = ticker_symbol
            
            if period == 'annually':
                # Filter for the last 4 years
                four_years_ago = pd.Timestamp.now() - pd.DateOffset(years=4)
                balance_sheet = balance_sheet[balance_sheet['Date'] >= four_years_ago]
            
            all_data.append(balance_sheet)

    # Concatenate all DataFrames into a single DataFrame
    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        
        # Include only the specified columns
        columns_to_include = ['Date', 'Symbol', 'Ordinary Shares Number', 'Share Issued',
                              'Tangible Book Value', 'Invested Capital', 'Working Capital',
                              'Net Tangible Assets', 'Common Stock Equity', 'Total Capitalization',
                              'Total Equity Gross Minority Interest', 'Stockholders Equity',
                              'Capital Stock', 'Common Stock',
                              'Total Liabilities Net Minority Interest',
                              'Total Non Current Liabilities Net Minority Interest',
                              'Current Liabilities', 'Payables', 'Accounts Payable', 'Total Assets',
                              'Total Non Current Assets', 'Other Non Current Assets', 'Net PPE',
                              'Accumulated Depreciation', 'Gross PPE',
                              'Machinery Furniture Equipment', 'Properties', 'Current Assets',
                              'Inventory', 'Accounts Receivable',
                              'Cash Cash Equivalents And Short Term Investments',
                              'Cash And Cash Equivalents']
        result_df = result_df[columns_to_include]
        
        # Reorder columns to have Date and Symbol first
        cols = ['Date', 'Symbol'] + [col for col in result_df.columns if col not in ['Date', 'Symbol']]
        result_df = result_df[cols]
        
        # Sort by Date (descending) and Symbol
        result_df = result_df.sort_values(['Symbol', 'Date'], ascending=[True, False])
    else:
        result_df = pd.DataFrame()

    return result_df

In [6]:
def get_cashflow(tickers, period='annually'):
    """
    Fetch cash flow data for a list of tickers using yfinance.

    Args:
    tickers (str or list): A single stock ticker symbol or a list of stock ticker symbols.
    period (str): 'annually' or 'quarterly' to fetch annual or quarterly cash flow data.

    Returns:
    pandas.DataFrame: A DataFrame containing the cash flow data for all tickers.
    """

    all_data = []

    # Ensure tickers is a list
    if isinstance(tickers, str):
        tickers = [tickers]

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        if period == 'annually':
            cashflow = ticker.cashflow if ticker.cashflow is not None else pd.DataFrame()
        elif period == 'quarterly':
            cashflow = ticker.quarterly_cashflow if ticker.quarterly_cashflow is not None else pd.DataFrame()
        else:
            raise ValueError("Invalid period. Use 'annually' or 'quarterly'.")

        if not cashflow.empty:
            # Standardize date format
            if period == 'annually':
                cashflow.columns = pd.to_datetime(cashflow.columns).to_period('Y').to_timestamp('Y')
            else:
                cashflow.columns = pd.to_datetime(cashflow.columns).to_period('Q').to_timestamp('Q')
            
            # Transpose the DataFrame
            cashflow = cashflow.T
            
            # Reset index to make date a column
            cashflow.reset_index(inplace=True)
            cashflow.rename(columns={'index': 'Date'}, inplace=True)
            
            # Add ticker column
            cashflow['Symbol'] = ticker_symbol
            
            if period == 'annually':
                # Filter for the last 4 years
                four_years_ago = pd.Timestamp.now() - pd.DateOffset(years=4)
                cashflow = cashflow[cashflow['Date'] >= four_years_ago]
            
            all_data.append(cashflow)

    # Concatenate all DataFrames into a single DataFrame
    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        
        # Include only the specified columns
        columns_to_include = ['Date', 'Symbol', 'Free Cash Flow', 'Capital Expenditure',
                              'End Cash Position', 'Beginning Cash Position', 'Changes In Cash',
                              'Financing Cash Flow', 'Net Issuance Payments Of Debt',
                              'Investing Cash Flow', 'Net PPE Purchase And Sale', 'Purchase Of PPE',
                              'Operating Cash Flow', 'Change In Working Capital',
                              'Change In Inventory', 'Change In Receivables', 'Other Non Cash Items',
                              'Depreciation And Amortization',
                              'Net Income From Continuing Operations']
        result_df = result_df[columns_to_include]
        
        # Reorder columns to have Date and Symbol first
        cols = ['Date', 'Symbol'] + [col for col in result_df.columns if col not in ['Date', 'Symbol']]
        result_df = result_df[cols]
        
        # Sort by Date (descending) and Symbol
        result_df = result_df.sort_values(['Symbol', 'Date'], ascending=[True, False])
    else:
        result_df = pd.DataFrame()

    return result_df

In [29]:
annually_income_statement_df = get_income_statement(tickers = tickers.tickers, period = 'annually')
annually_balance_sheet_df = get_balance_sheet(tickers = tickers.tickers, period = 'annually')
annually_cash_flow_df = get_cashflow(tickers = tickers.tickers, period = 'annually')

quarterly_income_statement_df = get_income_statement(tickers = tickers.tickers, period = 'quarterly')
quarterly_balance_sheet_df = get_balance_sheet(tickers = tickers.tickers, period = 'quarterly')
quarterly_cash_flow_df = get_cashflow(tickers = tickers.tickers, period = 'quarterly')

In [30]:
annually_income_statement_df

,Date,Symbol,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,...,Net Non Operating Interest Income Expense,Interest Expense Non Operating,Operating Income,Operating Expense,Research And Development,Selling General And Administration,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
36,2024-12-31,AAPL,0.0,0.241,134661000000.0,93736000000.0,11445000000.0,210352000000.0,134661000000.0,123216000000.0,...,NaN,NaN,123216000000.0,57467000000.0,31370000000.0,26097000000.0,180683000000.0,210352000000.0,391035000000.0,391035000000.0
37,2023-12-31,AAPL,0.0,0.147,125820000000.0,96995000000.0,11519000000.0,214137000000.0,125820000000.0,114301000000.0,...,-183000000.0,3933000000.0,114301000000.0,54847000000.0,29915000000.0,24932000000.0,169148000000.0,214137000000.0,383285000000.0,383285000000.0
38,2022-12-31,AAPL,0.0,0.162,130541000000.0,99803000000.0,11104000000.0,223546000000.0,130541000000.0,119437000000.0,...,-106000000.0,2931000000.0,119437000000.0,51345000000.0,26251000000.0,25094000000.0,170782000000.0,223546000000.0,394328000000.0,394328000000.0
39,2021-12-31,AAPL,0.0,0.133,123136000000.0,94680000000.0,11284000000.0,212981000000.0,123136000000.0,111852000000.0,...,198000000.0,2645000000.0,108949000000.0,43887000000.0,21914000000.0,21973000000.0,152836000000.0,212981000000.0,365817000000.0,365817000000.0
0,2024-12-31,ALC,-5481000.0,0.189,2705000000.0,1018000000.0,1228000000.0,4399000000.0,2676000000.0,1448000000.0,...,-120000000.0,192000000.0,1413000000.0,4099000000.0,876000000.0,3250000000.0,5512000000.0,4399000000.0,9911000000.0,9836000000.0
1,2023-12-31,ALC,-4420000.0,0.085,2294000000.0,974000000.0,1221000000.0,4208000000.0,2242000000.0,1021000000.0,...,-155000000.0,189000000.0,1039000000.0,4208000000.0,828000000.0,3209000000.0,5247000000.0,4208000000.0,9455000000.0,9370000000.0
2,2022-12-31,ALC,-21804000.0,0.276,1799000000.0,335000000.0,1123000000.0,3969000000.0,1720000000.0,597000000.0,...,-130000000.0,134000000.0,672000000.0,4076000000.0,702000000.0,3068000000.0,4748000000.0,3969000000.0,8717000000.0,8654000000.0
3,2021-12-31,ALC,-3500000.0,0.1,1792000000.0,376000000.0,1219000000.0,3639000000.0,1757000000.0,538000000.0,...,-127000000.0,120000000.0,580000000.0,4072000000.0,842000000.0,3076000000.0,4652000000.0,3639000000.0,8291000000.0,8222000000.0
8,2024-12-31,BLCO,-5700000.0,0.15,639000000.0,-317000000.0,436000000.0,1724000000.0,601000000.0,165000000.0,...,-384000000.0,399000000.0,188000000.0,2731000000.0,361000000.0,2082000000.0,2919000000.0,1872000000.0,4791000000.0,7492000000.0
9,2023-12-31,BLCO,-40800000.0,0.4,601000000.0,-260000000.0,382000000.0,1500000000.0,499000000.0,117000000.0,...,-268000000.0,283000000.0,204000000.0,2300000000.0,324000000.0,1736000000.0,2504000000.0,1642000000.0,4146000000.0,5899000000.0


In [31]:
quarterly_income_statement_df

,Date,Symbol,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,...,Net Non Operating Interest Income Expense,Interest Expense Non Operating,Operating Income,Operating Expense,Research And Development,Selling General And Administration,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
59,2024-12-31,AAPL,0.0,0.21,45912000000.0,36330000000.0,3080000000.0,66025000000.0,45912000000.0,42832000000.0,...,NaN,NaN,42832000000.0,15443000000.0,8268000000.0,7175000000.0,58275000000.0,66025000000.0,124300000000.0,124300000000.0
60,2024-09-30,AAPL,0.0,0.21,32502000000.0,14736000000.0,2911000000.0,51051000000.0,32502000000.0,29591000000.0,...,NaN,NaN,29591000000.0,14288000000.0,7765000000.0,6523000000.0,43879000000.0,51051000000.0,94930000000.0,94930000000.0
61,2024-06-30,AAPL,0.0,0.159,28202000000.0,21448000000.0,2850000000.0,46099000000.0,28202000000.0,25352000000.0,...,NaN,NaN,25352000000.0,14326000000.0,8006000000.0,6320000000.0,39678000000.0,46099000000.0,85777000000.0,85777000000.0
62,2024-03-31,AAPL,0.0,0.158,30736000000.0,23636000000.0,2836000000.0,48482000000.0,30736000000.0,27900000000.0,...,NaN,NaN,27900000000.0,14371000000.0,7903000000.0,6468000000.0,42271000000.0,48482000000.0,90753000000.0,90753000000.0
63,2023-12-31,AAPL,0.0,0.21,43221000000.0,33916000000.0,2848000000.0,64720000000.0,43221000000.0,40373000000.0,...,NaN,NaN,40373000000.0,14482000000.0,7696000000.0,6786000000.0,54855000000.0,64720000000.0,119575000000.0,119575000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,2024-06-30,RXST,0.0,0.21,-5237000.0,-6079000.0,820000.0,10637000.0,-5237000.0,-6057000.0,...,2270000.0,6000.0,-8333000.0,32583000.0,8291000.0,24292000.0,24250000.0,10637000.0,34887000.0,34887000.0
23,2024-03-31,RXST,0.0,0.21,-7945000.0,-9100000.0,1140000.0,8827000.0,-7945000.0,-9085000.0,...,1579000.0,6000.0,-10670000.0,31355000.0,8031000.0,23324000.0,20685000.0,8827000.0,29512000.0,29512000.0
24,2023-12-31,RXST,0.0,0.001958,-8107000.0,-9177000.0,1084000.0,10926000.0,-8107000.0,-9191000.0,...,1657000.0,4000.0,-10852000.0,28506000.0,7341000.0,21165000.0,17654000.0,10926000.0,28580000.0,28580000.0
25,2023-09-30,RXST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transform

In [33]:
def add_timeframe_column(df, timeframe):
    """
    Add a 'Time Frame' column to the DataFrame with the specified value.

    Args:
    df (pandas.DataFrame): The input DataFrame.
    timeframe (str): The value to be added in the 'Time Frame' column.

    Returns:
    pandas.DataFrame: The DataFrame with the new 'Time Frame' column.
    """
    df['Time Frame'] = timeframe
    return df

In [34]:
annually_income_statement_df = add_timeframe_column(annually_income_statement_df, "Annually")
annually_balance_sheet_df = add_timeframe_column(annually_balance_sheet_df, "Annually")
annually_cash_flow_df = add_timeframe_column(annually_cash_flow_df, "Annually")

quarterly_income_statement_df = add_timeframe_column(quarterly_income_statement_df, "Quarterly")
quarterly_balance_sheet_df = add_timeframe_column(quarterly_balance_sheet_df, "Quarterly")
quarterly_cash_flow_df = add_timeframe_column(quarterly_cash_flow_df, "Quarterly")

In [35]:
def add_quarter_and_year_columns(df):
    """
    Add 'Quarter' and 'Year' columns to the DataFrame.

    Args:
    df (pandas.DataFrame): The input DataFrame.

    Returns:
    pandas.DataFrame: The DataFrame with the new 'Quarter' and 'Year' columns.
""" 
    # Convert the 'date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract the quarter and year from the 'date' column
    df['Quarter'] = df['Date'].dt.quarter
    df['Year'] = df['Date'].dt.year
    
    return df

In [36]:
quarterly_income_statement_df = add_quarter_and_year_columns(quarterly_income_statement_df)
quarterly_balance_sheet_df = add_quarter_and_year_columns(quarterly_balance_sheet_df)
quarterly_cash_flow_df = add_quarter_and_year_columns(quarterly_cash_flow_df)

## Load

In [43]:
current_dir = os.getcwd()

# Change directory into data directory
data_dir = os.path.join(current_dir, 'datasets')

In [41]:
def load_data(df, data_dir, file_name):
    """
    Load a DataFrame to a CSV file in the specified data directory.

    Args:
    df (pandas.DataFrame): The DataFrame to be saved.
    data_dir (str): The directory where the CSV file will be saved.
    file_name (str): The name of the file to be saved (without extension).
    """

    # Create the full file path
    file_path = os.path.join(data_dir, f"{file_name}.csv")

    # Save the DataFrame to a CSV file
    df.to_csv(file_path, index=False)

In [44]:
# Load the annually income statement data
load_data(annually_income_statement_df, data_dir, 'annually_income_statement')

# Load the annually balance sheet data
load_data(annually_balance_sheet_df, data_dir, 'annually_balance_sheet')

# Load the annually cash flow statement data
load_data(annually_cash_flow_df, data_dir, 'annually_cash_flow')

# Load the quarterly income statement data
load_data(quarterly_income_statement_df, data_dir, 'quarterly_income_statement')

# Load the quarterly balance sheet data
load_data(quarterly_balance_sheet_df, data_dir, 'quarterly_balance_sheet')

# Load the quarterly cash flow statement data
load_data(quarterly_cash_flow_df, data_dir, 'quarterly_cash_flow')